In [5]:
import cv2 as cv
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time


class DrowsinessDetector:
    def __init__(self) -> None:
        # 1. mixer
        mixer.init()
        self.sound = mixer.Sound('alarm.wav')

        # 2. camera
        self.cam = cv.VideoCapture(0)
        self.font = cv.FONT_HERSHEY_COMPLEX_SMALL
        self.label=None

        # 3. model
        self.model = load_model('models/cnncat2.h5')
        self.score = 0 
        self.rpred=[99]
        self.lpred=[99]
        self.r_eye_label = None # can be Open Closed
        self.l_eye_label = None
        self.thicc = 2 
        
        #4. three classifiers
        self.face_classifier = cv.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
        self.leye_classifier = cv.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
        self.reye_classifier = cv.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

    def process_eyes(self, eye):
        eye = cv.cvtColor(eye,cv.COLOR_BGR2GRAY)
        eye = cv.resize(eye,(24,24))
        eye= eye/255
        eye=  eye.reshape(24,24,-1)
        eye = np.expand_dims(eye,axis=0)

        return eye

    def predict(self, eye):
        axis = 1

        prediction=self.model.predict(eye) 
        prediction=np.argmax(prediction,axis=axis)

        if(prediction[0]==1):
            self.label='Open' 
        if(prediction[0]==0):
            self.label='Closed'
        
        return self.label

    
    def __monitor(self):
        while(True):
            ret, frame = self.cam.read()
            height,width = frame.shape[:2] 

            gray_image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            
            faces = self.face_classifier.detectMultiScale(gray_image,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
            left_eye = self.leye_classifier.detectMultiScale(gray_image)
            right_eye =  self.reye_classifier.detectMultiScale(gray_image)

            cv.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv.FILLED )

            for (x,y,w,h) in faces:
                cv.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

                
    
            for (x,y,w,h) in right_eye:
                r_eye=frame[y:y+h,x:x+w]
                r_eye = self.process_eyes(r_eye)
                self.r_eye_label = self.predict(r_eye)
                
                break

            for (x,y,w,h) in left_eye:
                l_eye=frame[y:y+h,x:x+w]
                l_eye = self.process_eyes(l_eye)
                self.l_eye_label = self.predict(l_eye)

                break

            if self.r_eye_label == "Closed" and self.l_eye_label == "Closed":
                self.score=self.score+1
                cv.putText(frame,"Closed",(10,height-20), self.font, 1,(255,255,255),1,cv.LINE_AA)
            
            else:
                self.score=self.score-2
                cv.putText(frame,"Open",(10,height-20), self.font, 1,(255,255,255),1,cv.LINE_AA)
            
            self.score = 0 if self.score < 0 else self.score

            cv.putText(frame,'Score:'+str(self.score),(100,height-20), self.font, 1,(255,255,255),1,cv.LINE_AA)
            
            if(self.score>15):
                #person is feeling sleepy so we beep the alarm
                cv.imwrite(os.path.join(os.getcwd(),'image.jpg'),frame)
                try:
                    self.sound.play()
                    
                except:  # isplaying = False
                    pass

                if(self.thicc<16):
                    self.thicc= self.thicc+2
                else:
                    self.thicc=self.thicc-2
                    
                    if(self.thicc<2):
                        self.thicc=2
                        
                cv.rectangle(frame,(0,0),(width,height),(0,0,255),self.thicc) 
                
            cv.imshow('frame',frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                self.stop()
                break
    
    def monitor(self):
        try:
            self.__monitor()
        except Exception as e:
            self.stop()
            raise e

    def stop(self):
        self.sound.stop()
        self.cam.release()
        cv.destroyAllWindows()


In [6]:
DrowsinessDetector().monitor()

1/1 [==============================] - 0s 20ms/step
